In [1]:
import pandas as pd
from data import data_reader, TGDataset
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import torch
import torch.nn as nn
from PIL import Image
from torch.utils.data import DataLoader
import numpy as np
from torchvision import models, transforms
import pretrainedmodels
import matplotlib.pyplot as plt

Tensor = torch.Tensor
rand_m = np.random.random
PATH = 'test1000/'

In [2]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
trm = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    normalize
])

x_norm = transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                  std=[0.5, 0.5, 0.5])

val_trm = transforms.Compose([
    transforms.Resize(333),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    x_norm
])

def im_read(path):
    img = Image.open(path)
    return img.convert('RGB')

In [3]:
class args:
    bs = 128
    path = 'train2000/'
dr = data_reader(args, 'train2000/train2000.csv')

In [4]:
df = pd.read_csv('test1000.csv', header=None)

In [5]:
class model_xception(nn.Module):
    def __init__(self, num_class):
        super(model_xception, self).__init__()
        self.xception = pretrainedmodels.models.xception()
        self.avgp = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(2048, num_class)
    
    def forward(self, x):
        x = self.xception.features(x)
        x = self.avgp(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [6]:
model = model_xception(6)
model.load_state_dict(torch.load('weights/xception/best_params_acc97.75.pth'))
model.eval()

model_xception(
  (xception): Xception(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (block1): Block(
      (skip): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (skipbn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (rep): Sequential(
        (0): SeparableConv2d(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
          (pointwise): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace)


In [7]:
sub = pd.DataFrame()

In [8]:
len(df)

1000

In [ ]:
for i in range(len(df)):
    im = im_read(PATH + df[0][i])
    x = model(val_trm(im).unsqueeze(0))
    c = torch.max(x, 1)[1].item()
    sub = sub.append([(df[0][i], dr.mlb.classes_[c])])
    if i % 10 == 0: print(i)

0
10
20
30
40
50
60
70


In [ ]:
len(sub)

In [ ]:
sub.to_csv('submission.csv', header=None, index=None, encoding='utf-8')